In [1]:
import torch
import numpy as np
import pynq
from pynq import Overlay

from pynq import MMIO
from pynq import allocate
from pynq import Interrupt
import asyncio
import sys
import time
import math
import csv

In [2]:
# Configure the hardware 
overlay = Overlay('FEATHER.bit') # 
#overlay = Overlay('FEATHER.bit') # 
top_ip = overlay.QCONV_0
top_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, RESERVED_3=0, RESERVED_4=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  ifc1_offset_1 = Register(ifc1_offset=write-only),
  ifc1_offset_2 = Register(ifc1_offset=write-only),
  ifc2_offset_1 = Register(ifc2_offset=write-only),
  ifc2_offset_2 = Register(ifc2_offset=write-only),
  ifc3_offset_1 = Register(ifc3_offset=write-only),
  ifc3_offset_2 = Register(ifc3_offset=write-only),
  ifc4_offset_1 = Register(ifc4_offset=write-only),
  ifc4_offset_2 = Register(ifc4_offset=write-only),
  ifc5_offset_1 = Register(ifc5_offset=write-only),
  ifc5_offset_2 = Register(ifc5_offset=write-only),
  ifc6_offset_1 = Register(ifc6_offset=write-only),
  ifc6_offset_2 = Register(ifc6_offset=write-only),
  ifc7_1 = Register(ifc7=write-only),
  ifc7_2 = Regis

In [3]:
layer_idx_target_subnet_to_supernet = [[0, 1, 3, 4, 5, 6, 7, 8, 9, 16, 17, 18, 19, 20, 21, 22, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 48, 49, 50, 51, 52, 53, 54],
[0, 1, 3, 4, 5, 6, 7, 8, 9, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57],
[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57],
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]]

# top parameters
DEBUG_MODE = False
USE_DUMMY_DATA = False
INITIAL_OUTPUT_BUFFER = False 
USE_PARIAL_LAYER_DATA = False
USE_PERSISTENT_BUFFER = False
MAX_NUM_LAYER = 0
TEST_SPECIFIC_NETWORK_LAYER = False
for i in range(len(layer_idx_target_subnet_to_supernet)):
    num_layer_workload = len(layer_idx_target_subnet_to_supernet[i]) 
    MAX_NUM_LAYER = num_layer_workload if (num_layer_workload > MAX_NUM_LAYER) else MAX_NUM_LAYER
    
PRINT_UNIFIED_OUTPUT_BUFFER = True
PRINT_UNIFIED_IN_BUFFER = False
PRINT_COMPILED_DATA = False
PRINT_RECEIVED_OACTS = False

TOTAL_NUM_MODEL = 6
PARPLLELISM = [16, 9, 1] # K, C, Y
latency_per_layer_inference = []
dummy_n, dummy_k, dummy_c, dummy_r, dummy_s, dummy_x, dummy_y =  1, 24, 24, 3, 3, 20, 20

# Read the external file from outside data.

In [4]:
def convert_fix32_8_into_int8(scale_fp, output_size):
    decimal_bitwidth_fixed_point = 24
    out_scale_fp_scale_up_int = np.zeros(output_size, dtype="int8")
    size_in = len(scale_fp)
    for data_idx in range((output_size>>2)):
        temp_scale_int = np.round(scale_fp[data_idx] * (2**decimal_bitwidth_fixed_point)) if(data_idx < size_in) else 0
        temp_scale_int = int(temp_scale_int)

        out_scale_fp_scale_up_int[(data_idx<<2)]   =  temp_scale_int        & 0xff
        out_scale_fp_scale_up_int[(data_idx<<2)+1] = (temp_scale_int >> 8)  & 0xff
        out_scale_fp_scale_up_int[(data_idx<<2)+2] = (temp_scale_int >> 16) & 0xff
        out_scale_fp_scale_up_int[(data_idx<<2)+3] = (temp_scale_int >> 24) & 0xff

    return out_scale_fp_scale_up_int

def padding_int8_array_with_specific_zp(int8_data, output_size, specified_zero=None):
    """ 
        default output data type is int32, so every int32 demands 4 int8 
    """
    padding_zero = specified_zero if(specified_zero!=None) else 0 

    delta_size = output_size - int8_data.size
    if delta_size == 0:
        return int8_data
    else:
        int8_padding_data = np.full(delta_size, specified_zero, dtype="int8")
        int8_output_data = np.concatenate((int8_data, int8_padding_data), axis=0)
        return int8_output_data

class QCONV():
    """
        The overall iacts, weights are all stored inside separate buffer.
    """
    
    def __init__ (self, csv_data_path, bitstream_path, parallelism, top_ip):
        # Fuction control knob
        self.FIX_T_K_16 = True
        
        # FPGA Kernel
        self.top_ip = top_ip

        # Internal Parameters
        self.NUM_BYTE_HW_IFC = 16
        self.NUMBER_IACT_DATA_PER_UB_ENTRY = 16
        self.TRIPLE_NUM_HP_IFC = 18 # 3 cycle transfers with 6 interfaces being transferred every cycle.
        self.NUM_BYTE_IACT = 1
        self.NUM_BYTE_WEIGHT = 1
        self.NUM_IACTS = 16
        self.NUM_IACTS_PER_OFF_CHIP_IFC = math.ceil(self.NUM_BYTE_HW_IFC / self.NUM_BYTE_IACT)
        self.NUM_WEIGHTS_PER_OFF_CHIP_IFC = math.ceil(self.NUM_BYTE_HW_IFC / self.NUM_BYTE_WEIGHT)
        self.LOG_PAR_NUM_IACTS = 4
        self.NUMBER_WEIGHTS_DATA_PER_UB_ENTRY = 9

        # External Arguments
        self.PARALLEL_K = parallelism[0] # 16
        self.PARALLEL_C = parallelism[1] # 9
        self.PARALLEL_Y = parallelism[2] # 1
        self.iacts = None
        self.weights = None
        self.oacts = None
        self.overall_scale = None
        self.weights_zp = None
        self.reordered_iacts = None
        self.expected_oacts = None
        self.merged_expected_oacts = None

        self.csv_data_path = csv_data_path
        self.bitstream_path = bitstream_path
        self.num_reordered_iacts = 0
        self.padding_x = 0
        self.padding_y = 0
        self.stride_x = 0
        self.stride_y = 0
        self.dilation_x = 0
        self.dilation_y = 0
        self.iacts_zp_int = 0
        self.iacts_scale = 0
        self.oacts_zp_int = 0
        self.oacts_scale = 0
        self.N = 0
        self.K = 0
        self.C = 0
        self.X = 0
        self.Y = 0
        self.R = 0
        self.S = 0
        self.O_X = 0
        self.O_Y = 0
        
        self.mimic_N = None
        self.mimic_K = None
        self.mimic_C = None
        self.mimic_X = None
        self.mimic_Y = None
        self.mimic_R = None
        self.mimic_S = None
        self.mimic_O_X = None
        self.mimic_O_Y = None
        
        self.num_overall_scale_after_padding_hw_ifc = 0
        self.num_weights_zp_after_padding_hw_ifc = 0
        self.num_weights_after_padding_hw_ifc = 0
        self.num_iacts_after_padding_hw_ifc = 0
        self.unified_in_buffer = None
        self.unified_out_buffer = None

        self.num_reordered_iacts = None
        self.num_iacts_ifc_entries = None
        self.reordered_iacts = None

        self.total_compiled_data = None

        self.MAX_KERNEL_BUF_ENTRY_HALF = 32768
        
    def software_compile_fpga(self):
        self.top_ip.register_map.N = int(self.N)
        self.top_ip.register_map.K = int(self.K)
        self.top_ip.register_map.C = int(self.C)
        self.top_ip.register_map.R = int(self.R)
        self.top_ip.register_map.S = int(self.S)
        self.top_ip.register_map.X = int(self.X)
        self.top_ip.register_map.Y = int(self.Y)
        self.top_ip.register_map.padding_x = int(self.padding_x)
        self.top_ip.register_map.padding_y = int(self.padding_y)
        self.top_ip.register_map.stride_x = int(self.stride_x)
        self.top_ip.register_map.stride_y = int(self.stride_y)
        self.top_ip.register_map.dilation_x = int(self.dilation_x)
        self.top_ip.register_map.dilation_y = int(self.dilation_y)

        self.top_ip.write(0xcc, int(self.iacts_zp_int))
        self.top_ip.write(0xd4, int(self.oacts_zp_int))

        init_addr_oacts = self.num_overall_scale_after_padding_hw_ifc + self.num_weights_zp_after_padding_hw_ifc + self.num_weights_after_padding_hw_ifc + self.num_iacts_after_padding_hw_ifc
        num_k_ceiling_parallel = math.ceil(self.K / self.PARALLEL_K)
        num_iter_oacts = num_k_ceiling_parallel*self.O_X*self.O_Y

        self.unified_in_buffer = pynq.allocate((init_addr_oacts << 4), "int8")
        print(f"number in buffer entries = {(init_addr_oacts << 4)}, dtype={self.unified_in_buffer.dtype}")
        
        self.unified_out_buffer = pynq.allocate((num_iter_oacts << 4), dtype=np.uint8)
        print(f"number output buffer entries = {(num_iter_oacts << 4)}")
        
        self.top_ip.write(0xdc, (0))

        self.unified_in_buffer.freebuffer()
        self.unified_in_buffer[:] = self.total_compiled_data[:]
        
        if PRINT_UNIFIED_IN_BUFFER:
            for _idx in range(self.unified_in_buffer.size):
                print(self.unified_in_buffer[_idx])
        
        self.unified_in_buffer.sync_to_device()
    
        self.top_ip.write(0x10, self.unified_in_buffer.physical_address & 0xFFFFFFFF )
        self.top_ip.write(0x14, self.unified_in_buffer.physical_address >> 32)
        self.top_ip.write(0x1c, self.unified_in_buffer.physical_address & 0xFFFFFFFF )
        self.top_ip.write(0x20, self.unified_in_buffer.physical_address >> 32)
        self.top_ip.write(0x28, self.unified_in_buffer.physical_address & 0xFFFFFFFF )
        self.top_ip.write(0x2c, self.unified_in_buffer.physical_address >> 32)
        self.top_ip.write(0x34, self.unified_in_buffer.physical_address & 0xFFFFFFFF )
        self.top_ip.write(0x38, self.unified_in_buffer.physical_address >> 32)
        self.top_ip.write(0x40, self.unified_in_buffer.physical_address & 0xFFFFFFFF )
        self.top_ip.write(0x44, self.unified_in_buffer.physical_address >> 32)
        self.top_ip.write(0x4c, self.unified_in_buffer.physical_address & 0xFFFFFFFF )
        self.top_ip.write(0x50, self.unified_in_buffer.physical_address >> 32)
        self.top_ip.write(0x58, self.unified_out_buffer.physical_address & 0xFFFFFFFF )
        self.top_ip.write(0x5c, self.unified_out_buffer.physical_address >> 32)
        
    def run_fpga_profile(self):
        total_number_testing = 200
        start_time = time.time()
        for i in range(total_number_testing):
            self.top_ip.register_map.CTRL.AP_START = 1

            # get it start
            while(int(self.top_ip.register_map.CTRL.AP_IDLE) == 0):
                pass

        end_time = time.time()
        return (end_time - start_time)/total_number_testing
    
    def data_read_from_external_file(self):
        csv_parameters_file = open(f"{self.csv_data_path}/parameter.csv", "r") 
        csv_parameters_data = csv.reader(csv_parameters_file, delimiter="," )
        
        iacts_file = open(f"{self.csv_data_path}/iacts.csv", "r") 
        iacts_data = csv.reader(iacts_file, delimiter="," )

        weights_file = open(f"{self.csv_data_path}/weights.csv", "r") 
        weights_data = csv.reader(weights_file, delimiter="," )
        
        oacts_file = open(f"{self.csv_data_path}/oacts.csv", "r")
        oacts_data = csv.reader(oacts_file, delimiter="," )
        
        weights_parameter_file = open(f"{self.csv_data_path}/weights_quant_parameters.csv", "r")
        weights_parameter_data = csv.reader(weights_parameter_file, delimiter="," )
        
        for row in csv_parameters_data:
            self.padding_x = int(row[0])
            self.padding_y = int(row[1])
            self.stride_x = int(row[2])
            self.stride_y = int(row[3])
            self.dilation_x = int(row[4])
            self.dilation_y = int(row[5])
            self.iacts_zp_int = int(row[6])
            self.iacts_scale = float(row[7])
            self.oacts_zp_int = int(row[8])
            self.oacts_scale = float(row[9])
            self.N = int(row[10])
            self.K = int(row[11])
            self.C = int(row[12])
            self.X = int(row[13])
            self.Y = int(row[14])
            self.R = int(row[15])
            self.S = int(row[16])
            self.O_X = int(row[17])
            self.O_Y = int(row[18])

        print(f"padding_x={self.padding_x}")
        print(f"padding_y={self.padding_y}")
        print(f"stride_x={self.stride_x}")
        print(f"stride_y={self.stride_y}")
        print(f"dilation_x={self.dilation_x}")
        print(f"dilation_y={self.dilation_y}")
        print(f"iacts_zp_int={self.iacts_zp_int}")
        print(f"iacts_scale={self.iacts_scale}")
        print(f"oacts_zp_int={self.oacts_zp_int}")
        print(f"oacts_scale={self.oacts_scale}")
        print(f"N={self.N}")
        print(f"K={self.K}")
        print(f"C={self.C}")
        print(f"X={self.X}")
        print(f"Y={self.Y}")
        print(f"R={self.R}")
        print(f"S={self.S}")
        print(f"O_X={self.O_X}")
        print(f"O_Y={self.O_Y}")

        self.iacts = np.zeros([self.N*self.C*self.X*self.Y], dtype=np.uint8)
        self.weights = np.zeros([self.K*self.C*self.R*self.S], dtype=np.int8)
        self.oacts = np.zeros([self.N*self.K*self.O_X*self.O_Y], dtype=np.uint8)
        self.weights_zp = np.zeros([self.K], dtype=np.int8)
        self.overall_scale = np.zeros([self.K], dtype=np.float64)

        addr_idx = 0
        for row in iacts_data:
            for data in row:
                self.iacts[addr_idx] = int(data)
                addr_idx += 1

        addr_idx = 0
        for row in weights_data:
            for data in row:
                self.weights[addr_idx] = int(data)
                addr_idx += 1

        addr_idx = 0
        for row in oacts_data:
            for data in row:
                self.oacts[addr_idx] = int(data)
                addr_idx += 1

        for row in weights_parameter_data:
            for data_idx in range(len(row)>>1):
                self.weights_zp[data_idx] = int(row[data_idx])
                self.overall_scale[data_idx] = float(row[data_idx + (len(row)>>1)])
                
    def data_fed_directly_from_pytorch(self, iacts, weights, stride, padding, dilation, output_scale, output_zp):
        self.padding_x = padding[0]
        self.padding_y = padding[1]
        self.stride_x = stride[0]
        self.stride_y = stride[1]
        self.dilation_x = dilation[0]
        self.dilation_y = dilation[1]
        self.iacts_zp_int = iacts.q_zero_point()
        self.iacts_scale = iacts.q_scale()
        self.oacts_zp_int = output_zp
        self.oacts_scale = output_scale
        self.N = iacts.shape[0]
        self.K = weights.shape[0]
        self.C = weights.shape[1]
        self.X = iacts.shape[2]
        self.Y = iacts.shape[3]
        self.R = weights.shape[2]
        self.S = weights.shape[3]
        self.O_X = math.floor((iacts.shape[2] + 2 * padding[0] - dilation[0] * (weights.shape[2] - 1) - 1) / stride[0] + 1)
        self.O_Y = math.floor((iacts.shape[3] + 2 * padding[1] - dilation[1] * (weights.shape[3] - 1) - 1) / stride[1] + 1)
        self.iacts = iacts.int_repr().detach().numpy().flatten()
        self.weights_zp = weights.q_per_channel_zero_points().numpy()
        self.weights_scale = weights.q_per_channel_scales().numpy()
        self.weights = weights.int_repr().detach().numpy().flatten()

        print(f"padding_x={self.padding_x}")
        print(f"padding_y={self.padding_y}")
        print(f"stride_x={self.stride_x}")
        print(f"stride_y={self.stride_y}")
        print(f"dilation_x={self.dilation_x}")
        print(f"dilation_y={self.dilation_y}")
        print(f"iacts_zp_int={self.iacts_zp_int}")
        print(f"iacts_scale={self.iacts_scale}")
        print(f"oacts_zp_int={self.oacts_zp_int}")
        print(f"oacts_scale={self.oacts_scale}")
        print(f"N={self.N}")
        print(f"K={self.K}")
        print(f"C={self.C}")
        print(f"X={self.X}")
        print(f"Y={self.Y}")
        print(f"R={self.R}")
        print(f"S={self.S}")
        print(f"O_X={self.O_X}")
        print(f"O_Y={self.O_Y}")
        
        self.overall_scale = np.zeros(self.K, dtype=np.float32)
        for k in range(self.K):
            self.overall_scale[k] = np.divide(np.multiply(self.weights_scale[k], self.iacts_scale), self.oacts_scale)
        
    def reorder_iacts(self):
        ##############################################
        ### --- Input reorder to satisfy parallelism 
        ##############################################
        # need to r input to satisfy the parallelism requirement of the hardware
        PARALLEL_C = self.PARALLEL_C
        num_c_ceiling = math.ceil(self.C/PARALLEL_C)
        self.num_reordered_iacts = math.ceil(self.C/PARALLEL_C)*self.N*self.X*self.Y*PARALLEL_C
        self.num_iacts_ifc_entries = math.ceil(self.num_reordered_iacts/ self.NUM_IACTS_PER_OFF_CHIP_IFC)
        self.reordered_iacts = np.zeros(self.num_reordered_iacts, dtype=np.uint8)
        for n in range(self.N):
            for c in range(0, self.C, PARALLEL_C):
                for x in range(self.X):
                    for y in range(self.Y):
                        for cp_id in range(PARALLEL_C):
                            self.reordered_iacts[n*num_c_ceiling*self.X*self.Y*PARALLEL_C + int(c/PARALLEL_C)*self.X*self.Y*PARALLEL_C + x*self.Y*PARALLEL_C + y*PARALLEL_C + cp_id] = self.iacts[n*self.C*self.X*self.Y + (c + cp_id)*self.X*self.Y + x*self.Y + y] if (c + cp_id < self.C) else self.iacts_zp_int
        
    def print_in_data_info(self):
        print("overall scales are")
        for data_idx in range(len(self.overall_scale)):
            print(self.overall_scale[data_idx], end=" ")
        print()

        print("weights zero points are")
        for data_idx in range(len(self.weights_zp)):
            print(self.weights_zp[data_idx], end=" ")
        print()
        
        print("iacts data are")
        offset_n = self.C*self.X*self.Y
        offset_x = self.X*self.Y
        offset_y = self.Y
        for n in range(self.N):
            for c in range(self.C):
                for x in range(self.X):
                    for y in range(self.Y):
                        print(self.iacts[n*offset_n + c*offset_x + x*offset_y + y], end=" ")
                    print()
        print()

        print("weights data are")
        offset_weight_k = self.C*self.R*self.S
        offset_weight_c = self.R*self.S
        offset_weight_r = self.S
        for k in range(self.K):
            for c in range(self.C):
                for r in range(self.R):
                    for s in range(self.S):
                        print(self.weights[k*offset_weight_k + c*offset_weight_c + r*offset_weight_r + s], end=" ")
                    print()
        print()

        print("reordered iacts data are")
        offset_n = math.ceil(self.C / self.PARALLEL_C) * self.X * self.Y * self.PARALLEL_C
        offset_c = math.ceil(self.X * self.Y * self.PARALLEL_C)
        offset_x = math.ceil(self.Y * self.PARALLEL_C)
        if self.R == 3:
            for n in range(self.N):
                for c in range(0, self.C, self.PARALLEL_C):
                    for x in range(self.X):
                        for y in range(self.Y):
                            for cp_id in range(self.PARALLEL_C):
                                print(self.reordered_iacts[n*offset_n + c*offset_c + x*offset_x + y*self.PARALLEL_C + cp_id], end=" ")
                            print()
            print()
        else:
            for n in range(self.mimic_N):
                for c in range(0, self.mimic_C, self.PARALLEL_C):
                    for x in range(self.mimic_X):
                        for y in range(self.mimic_Y):
                            for cp_id in range(self.PARALLEL_C):
                                print(self.reordered_iacts_1x1[n*offset_n + c*offset_c + x*offset_x + y*self.PARALLEL_C + cp_id], end=" ")
                            print()
            print()

    def compile_data_feeding_hw(self):
        """
            This subfunction is for the 3x3 kernel only.
        """
        ##############################################
        ### --- using int32 as the transfer data
        ##############################################
        # padding zero to fill in all spaces in 128-bit interface.
        self.num_overall_scale_after_padding_hw_ifc =  math.ceil(len(self.overall_scale) / (self.NUM_BYTE_HW_IFC >> 2))
        self.num_weights_zp_after_padding_hw_ifc = math.ceil(len(self.weights_zp) /  self.NUM_BYTE_HW_IFC)
        self.num_weights_after_padding_hw_ifc = math.ceil(len(self.weights) / self.NUM_BYTE_HW_IFC)
        self.num_iacts_after_padding_hw_ifc = math.ceil(math.ceil(self.num_reordered_iacts / self.NUM_BYTE_HW_IFC) / self.TRIPLE_NUM_HP_IFC) * self.TRIPLE_NUM_HP_IFC

        print(f"len(overall_scale) = {len(self.overall_scale)}  number after padding 128-bit interface and parallel interfaces = {self.num_overall_scale_after_padding_hw_ifc}")
        print(f"len(weights_zp) = {len(self.weights_zp)}  number after padding 128-bit interface and parallel interfaces = {self.num_weights_zp_after_padding_hw_ifc}")
        print(f"len(weights) = {len(self.weights)}  number after padding 128-bit interface and parallel interfaces = {self.num_weights_after_padding_hw_ifc}")
        print(f"len(iacts) = {len(self.reordered_iacts)}  number after padding 128-bit interface and parallel interfaces = {self.num_iacts_after_padding_hw_ifc}")

        ### Allocate space for new input data
        compiled_data_overall_scale = convert_fix32_8_into_int8(self.overall_scale, (self.num_overall_scale_after_padding_hw_ifc << 4))
        compiled_data_weights_zp = padding_int8_array_with_specific_zp(self.weights_zp, (self.num_weights_zp_after_padding_hw_ifc << 4), self.weights_zp[0])
        compiled_data_iacts = padding_int8_array_with_specific_zp(self.reordered_iacts, (self.num_iacts_after_padding_hw_ifc << 4), self.iacts_zp_int)

        # calculate zero entries for weights padding.
        if self.FIX_T_K_16:
            T_K = 16
            total_iteration = math.ceil(self.K/ T_K)
        else:
            num_ub_entry_kernel                  = math.ceil((self.num_weights_after_padding_hw_ifc << 4)/ self.NUMBER_WEIGHTS_DATA_PER_UB_ENTRY)
            total_iteration                      = math.ceil(num_ub_entry_kernel / self.MAX_KERNEL_BUF_ENTRY_HALF)
            T_K_pre                              = math.ceil(self.K / total_iteration)
            T_K                                  = math.ceil(T_K_pre / self.PARALLEL_K) * self.PARALLEL_K

        new_weights_len = (T_K * total_iteration) * self.C * self.R * self.S
        print(f"self.K = {self.K}, T_K = {T_K}, overall_K post padding = {T_K * total_iteration}, total_iteration = {total_iteration}, overall weights length post-padding = {new_weights_len}")
        self.num_weights_after_padding_hw_ifc = math.ceil(new_weights_len / self.NUM_BYTE_HW_IFC)

        compiled_data_weights = padding_int8_array_with_specific_zp(self.weights, (self.num_weights_after_padding_hw_ifc << 4), self.weights_zp[0])

        self.total_compiled_data = np.concatenate((compiled_data_overall_scale, compiled_data_weights_zp, compiled_data_iacts, compiled_data_weights), axis=0)
        print(f"total_compiled_data.size = {self.total_compiled_data.size}")
        if PRINT_COMPILED_DATA:
            for _idx in range(self.total_compiled_data.size):
                print(self.total_compiled_data[_idx])

    def run_workload_on_fpga(self):
        """
            For 3x3 kernel only.
        """
        self.reorder_iacts()
        # qconv2d_instance.print_in_data_info()
        self.compile_data_feeding_hw()
        self.software_compile_fpga()
        latency = self.run_fpga_profile()
        print(f"latency = {latency}") 
        
        return latency

    def self_result_parser(self):
        self.unified_out_buffer.sync_from_device()
        self.unified_out_buffer.flush()
        if PRINT_RECEIVED_OACTS:
            for _idx in range(len(self.unified_out_buffer)):
                print(self.unified_out_buffer[_idx], end=" ")
        
        self.unified_out_buffer = self.unified_out_buffer.reshape([math.ceil(self.K/self.PARALLEL_K), self.O_X, self.O_Y, self.PARALLEL_K])
        self.unified_out_buffer = np.swapaxes(self.unified_out_buffer,2,3)
        self.unified_out_buffer = np.swapaxes(self.unified_out_buffer,1,2)
        self.unified_out_buffer = self.unified_out_buffer.reshape([math.ceil(self.K/self.PARALLEL_K)*self.PARALLEL_K, self.O_X, self.O_Y])
        self.unified_out_buffer = self.unified_out_buffer[0:self.K, :, :]
        print(f"finish reshape")

    def result_parser(self, unified_out_buffer):
        unified_out_buffer.sync_from_device()
        unified_out_buffer.flush()
        unified_out_buffer = unified_out_buffer.reshape([math.ceil(self.K/self.PARALLEL_K), self.O_X, self.O_Y, self.PARALLEL_K])
        unified_out_buffer = np.swapaxes(unified_out_buffer,2,3)
        unified_out_buffer = np.swapaxes(unified_out_buffer,1,2)
        unified_out_buffer = unified_out_buffer.reshape([math.ceil(self.K/self.PARALLEL_K)*self.PARALLEL_K, self.O_X, self.O_Y])
        unified_out_buffer = unified_out_buffer[0:self.K, :, :]
        return unified_out_buffer

In [5]:
if TEST_SPECIFIC_NETWORK_LAYER:
    TOTAL_NUM_MODEL = 1
    TOTAL_NUM_LAYER_WORKLOAD = 1

latency_table_layer = np.zeros((TOTAL_NUM_MODEL, MAX_NUM_LAYER), dtype=float)

for network_idx in range(TOTAL_NUM_MODEL):
    TOTAL_NUM_LAYER_WORKLOAD = len(layer_idx_target_subnet_to_supernet[network_idx])
    for layer_idx in range(TOTAL_NUM_LAYER_WORKLOAD):
    # for layer_idx in range(TOTAL_NUM_LAYER_WORKLOAD-init_file_id):
        print(f"executing network {network_idx} layer {layer_idx}")
        path_iacts = f"/home/xilinx/jupyter_notebooks/qconv2d/CPU_Kernel/conv_iActs_{network_idx}_{layer_idx}.pth"
        path_weights = f"/home/xilinx/jupyter_notebooks/qconv2d/CPU_Kernel/conv_weights_{network_idx}_{layer_idx}.pth"
        path_parameters = f"/home/xilinx/jupyter_notebooks/qconv2d/CPU_Kernel/conv_parameter_{network_idx}_{layer_idx}.pth"

        iacts = torch.load(path_iacts)
        weights = torch.load(path_weights)
        parameters = torch.load(path_parameters)

        [padding0, padding1, stride0, stride1, dilation0, dilation1, output_scale, output_zp] = parameters

        padding = [int(padding0), int(padding1)]
        stride = [int(stride0), int(stride1)]
        dilation = [int(dilation0), int(dilation1)]
        output_scale = float(output_scale)
        output_zp = int(output_zp)

        print(f"{iacts.shape[0]}, {weights.shape[0]}, {weights.shape[1]}, {weights.shape[2]}, {weights.shape[3]}, {iacts.shape[2]}, {iacts.shape[3]}, {padding[0]}, {padding[1]}, {stride[0]}, {stride[1]}, {dilation[0]}, {dilation[1]}, {output_zp}, {output_scale}")
        o_x = math.floor((iacts.shape[2] + 2*padding[0] - dilation[0] * (weights.shape[2] - 1) - 1) / stride[0] + 1)
        o_y = math.floor((iacts.shape[3] + 2*padding[1] - dilation[1] * (weights.shape[3] - 1) - 1) / stride[1] + 1)

        ################### uncomment the following lines if wanna use the dummy data.
        if USE_DUMMY_DATA or USE_PARIAL_LAYER_DATA:
            if USE_DUMMY_DATA:
                def generate_dummy_workload(n, k, c, r, s, x, y, iacts, weights):
                    weights_fp = torch.rand(weights.shape, torch.float32) - 0.5
                    iacts_fp = torch.rand(iacts.shape, dtype=torch.float32)
                    dummy_iacts = iacts_fp[0:n,0:c,0:x,0:y]
                    dummy_weights = weights_fp[0:k,0:c,0:r,0:s]
                    dummy_iacts = torch.quantize_per_tensor(dummy_iacts, iacts.q_scale(), iacts.q_zero_point(), torch.quint8)
                    dummy_weights = torch.quantize_per_channel(dummy_weights, weights.q_per_channel_scales()[0:k], weights.q_per_channel_zero_points()[0:k], 0, torch.qint8)
                    return dummy_iacts, dummy_weights

                dummy_iact, dummy_weights = generate_dummy_workload(dummy_n, dummy_k, dummy_c, dummy_r, dummy_s, dummy_x, dummy_y, iacts, weights)

                csv_data_path = ""
                bitstream_path = ""
                qconv2d_instance = QCONV(csv_data_path, bitstream_path, PARPLLELISM)
                qconv2d_instance.data_fed_directly_from_pytorch(dummy_iact, dummy_weights, stride, padding, dilation, output_scale, output_zp)

            elif USE_PARIAL_LAYER_DATA:
                print("partiall data")

                def generate_dummy_workload(n, k, c, r, s, x, y, iacts, weights):
                    print(f"dummy workload size = n={n}, k={k}, c={c}, r={r}, s={s}, x={x}, y={y}")
                    need_quantized_iacts = True 
                    if(n == iacts.shape[0] and c == iacts.shape[1] and x == iacts.shape[2] and y == iacts.shape[3]):
                        need_quantized_iacts = False

                    if(need_quantized_iacts):
                        iacts_fp = np.multiply(np.subtract(iacts.int_repr()[0:n, 0:c, 0:x, 0:y], float(iacts.q_zero_point())), float(iacts.q_scale()))
                        dummy_iacts = torch.quantize_per_tensor(iacts_fp.float(), iacts.q_scale(), iacts.q_zero_point(), torch.quint8)
                    else:
                        iacts_fp = iacts
                        dummy_iacts = iacts

                    weights_fp = torch.zeros([k,c,r,s], dtype=torch.float)
                    for _k in range(k):
                        weights_fp[_k] = np.multiply(np.subtract(weights.int_repr()[_k, 0:c, 0:r, 0:s], weights.q_per_channel_zero_points()[_k]), weights.q_per_channel_scales()[_k])                    

                    dummy_weights = torch.quantize_per_channel(weights_fp.float(), weights.q_per_channel_scales()[0:k], weights.q_per_channel_zero_points()[0:k], 0, torch.qint8)
                    return iacts_fp, weights_fp, dummy_iacts, dummy_weights

                _,_, dummy_iact, dummy_weights = generate_dummy_workload(dummy_n, dummy_k, dummy_c, dummy_r, dummy_s, dummy_x, dummy_y, iacts, weights)

                csv_data_path = ""
                bitstream_path = ""
                qconv2d_instance = QCONV(csv_data_path, bitstream_path, PARPLLELISM, top_ip)
                qconv2d_instance.data_fed_directly_from_pytorch(dummy_iact, dummy_weights, stride, padding, dilation, output_scale, output_zp)

            latency = qconv2d_instance.run_workload_on_fpga()
            qconv2d_instance.self_result_parser()
            latency_table_layer[network_idx, layer_idx] = latency
            print(f"latency = {latency}")

        else:
            """
                The workload might overflow the size of unified buffer.
            """
            if weights.shape[2]==1:
                latency_table_layer[network_idx, layer_idx] = 0
            else:
                csv_data_path = ""
                bitstream_path = ""
                qconv2d_instance = QCONV(csv_data_path, bitstream_path, PARPLLELISM, top_ip)
                qconv2d_instance.data_fed_directly_from_pytorch(iacts, weights, stride, padding, dilation, output_scale, output_zp)
                fpga_latency = qconv2d_instance.run_workload_on_fpga()
                qconv2d_instance.self_result_parser()
                latency_table_layer[network_idx, layer_idx] = fpga_latency

print("#### latency of all layers in different models###")
for i in range(latency_table_layer.shape[0]):
    for j in range(latency_table_layer.shape[1]):
        print(latency_table_layer[i,j], end=" ")
    print()
    
latency_table = np.sum(latency_table_layer, axis=1)
print("\n#### latency of different model###\n")
for i in range(latency_table.shape[0]):
    print(latency_table[i])

executing network 0 layer 0
1, 24, 3, 3, 3, 160, 160, 1, 1, 2, 2, 1, 1, 64, 0.4009031057357788
padding_x=1
padding_y=1
stride_x=2
stride_y=2
dilation_x=1
dilation_y=1
iacts_zp_int=57
iacts_scale=0.03740893304347992
oacts_zp_int=64
oacts_scale=0.4009031057357788
N=1
K=24
C=3
X=160
Y=160
R=3
S=3
O_X=80
O_Y=80
len(overall_scale) = 24  number after padding 128-bit interface and parallel interfaces = 6
len(weights_zp) = 24  number after padding 128-bit interface and parallel interfaces = 2
len(weights) = 648  number after padding 128-bit interface and parallel interfaces = 41
len(iacts) = 230400  number after padding 128-bit interface and parallel interfaces = 14400
self.K = 24, T_K = 16, overall_K post padding = 32, total_iteration = 2, overall weights length post-padding = 864
total_compiled_data.size = 231392
number in buffer entries = 231392, dtype=int8
number output buffer entries = 204800
latency = 0.0007022202014923095
finish reshape
executing network 0 layer 1
1, 40, 24, 3, 3, 80, 8

len(overall_scale) = 136  number after padding 128-bit interface and parallel interfaces = 34
len(weights_zp) = 136  number after padding 128-bit interface and parallel interfaces = 9
len(weights) = 166464  number after padding 128-bit interface and parallel interfaces = 10404
len(iacts) = 14400  number after padding 128-bit interface and parallel interfaces = 900
self.K = 136, T_K = 16, overall_K post padding = 144, total_iteration = 9, overall weights length post-padding = 176256
total_compiled_data.size = 191344
number in buffer entries = 191344, dtype=int8
number output buffer entries = 14400
latency = 0.0003100240230560303
finish reshape
executing network 0 layer 22
1, 664, 136, 1, 1, 10, 10, 0, 0, 1, 1, 1, 1, 67, 0.11305078864097595
executing network 0 layer 23
1, 136, 664, 1, 1, 10, 10, 0, 0, 1, 1, 1, 1, 58, 0.4851232171058655
executing network 0 layer 24
1, 136, 136, 3, 3, 10, 10, 1, 1, 1, 1, 1, 1, 72, 0.35597026348114014
padding_x=1
padding_y=1
stride_x=1
stride_y=1
dilation_x

latency = 0.0009643566608428955
finish reshape
executing network 1 layer 2
1, 32, 64, 1, 1, 40, 40, 0, 0, 1, 1, 1, 1, 57, 0.9004822969436646
executing network 1 layer 3
1, 32, 32, 3, 3, 40, 40, 1, 1, 1, 1, 1, 1, 60, 0.3900472819805145
padding_x=1
padding_y=1
stride_x=1
stride_y=1
dilation_x=1
dilation_y=1
iacts_zp_int=85
iacts_scale=0.3303258717060089
oacts_zp_int=60
oacts_scale=0.3900472819805145
N=1
K=32
C=32
X=40
Y=40
R=3
S=3
O_X=40
O_Y=40
len(overall_scale) = 32  number after padding 128-bit interface and parallel interfaces = 8
len(weights_zp) = 32  number after padding 128-bit interface and parallel interfaces = 2
len(weights) = 9216  number after padding 128-bit interface and parallel interfaces = 576
len(iacts) = 57600  number after padding 128-bit interface and parallel interfaces = 3600
self.K = 32, T_K = 16, overall_K post padding = 32, total_iteration = 2, overall weights length post-padding = 9216
total_compiled_data.size = 66976
number in buffer entries = 66976, dtype=int

padding_x=1
padding_y=1
stride_x=1
stride_y=1
dilation_x=1
dilation_y=1
iacts_zp_int=58
iacts_scale=0.4188968539237976
oacts_zp_int=73
oacts_scale=0.3705727159976959
N=1
K=208
C=208
X=10
Y=10
R=3
S=3
O_X=10
O_Y=10
len(overall_scale) = 208  number after padding 128-bit interface and parallel interfaces = 52
len(weights_zp) = 208  number after padding 128-bit interface and parallel interfaces = 13
len(weights) = 389376  number after padding 128-bit interface and parallel interfaces = 24336
len(iacts) = 21600  number after padding 128-bit interface and parallel interfaces = 1350
self.K = 208, T_K = 16, overall_K post padding = 208, total_iteration = 13, overall weights length post-padding = 389376
total_compiled_data.size = 412016
number in buffer entries = 412016, dtype=int8
number output buffer entries = 20800
latency = 0.0005568742752075195
finish reshape
executing network 1 layer 28
1, 816, 208, 1, 1, 10, 10, 0, 0, 1, 1, 1, 1, 63, 0.18971998989582062
executing network 1 layer 29
1, 16

latency = 0.001349184513092041
finish reshape
executing network 2 layer 2
1, 40, 64, 1, 1, 48, 48, 0, 0, 1, 1, 1, 1, 57, 1.0678623914718628
executing network 2 layer 3
1, 40, 40, 3, 3, 48, 48, 1, 1, 1, 1, 1, 1, 62, 0.4916747212409973
padding_x=1
padding_y=1
stride_x=1
stride_y=1
dilation_x=1
dilation_y=1
iacts_zp_int=85
iacts_scale=0.36717402935028076
oacts_zp_int=62
oacts_scale=0.4916747212409973
N=1
K=40
C=40
X=48
Y=48
R=3
S=3
O_X=48
O_Y=48
len(overall_scale) = 40  number after padding 128-bit interface and parallel interfaces = 10
len(weights_zp) = 40  number after padding 128-bit interface and parallel interfaces = 3
len(weights) = 14400  number after padding 128-bit interface and parallel interfaces = 900
len(iacts) = 103680  number after padding 128-bit interface and parallel interfaces = 6480
self.K = 40, T_K = 16, overall_K post padding = 48, total_iteration = 3, overall weights length post-padding = 17280
total_compiled_data.size = 121168
number in buffer entries = 121168, dty

...

len(overall_scale) = 256  number after padding 128-bit interface and parallel interfaces = 64
len(weights_zp) = 256  number after padding 128-bit interface and parallel interfaces = 16
len(weights) = 589824  number after padding 128-bit interface and parallel interfaces = 36864
len(iacts) = 51156  number after padding 128-bit interface and parallel interfaces = 3204
self.K = 256, T_K = 16, overall_K post padding = 256, total_iteration = 16, overall weights length post-padding = 589824
total_compiled_data.size = 642368
number in buffer entries = 642368, dtype=int8
number output buffer entries = 50176
latency = 0.0013235890865325927
finish reshape
executing network 4 layer 44
1, 1024, 256, 1, 1, 14, 14, 0, 0, 1, 1, 1, 1, 67, 0.05210597813129425
executing network 4 layer 45
1, 208, 1024, 1, 1, 14, 14, 0, 0, 1, 1, 1, 1, 59, 0.27035561203956604
executing network 4 layer 46
1, 208, 208, 3, 3, 14, 14, 1, 1, 1, 1, 1, 1, 58, 0.1405782401561737
padding_x=1
padding_y=1
stride_x=1
stride_y=1
dilat

latency = 0.0010275328159332275
finish reshape
executing network 5 layer 2
1, 64, 24, 3, 3, 112, 112, 1, 1, 1, 1, 1, 1, 70, 1.963769555091858
padding_x=1
padding_y=1
stride_x=1
stride_y=1
dilation_x=1
dilation_y=1
iacts_zp_int=0
iacts_scale=0.4390455186367035
oacts_zp_int=70
oacts_scale=1.963769555091858
N=1
K=64
C=24
X=112
Y=112
R=3
S=3
O_X=112
O_Y=112
len(overall_scale) = 64  number after padding 128-bit interface and parallel interfaces = 16
len(weights_zp) = 64  number after padding 128-bit interface and parallel interfaces = 4
len(weights) = 13824  number after padding 128-bit interface and parallel interfaces = 864
len(iacts) = 338688  number after padding 128-bit interface and parallel interfaces = 21168
self.K = 64, T_K = 16, overall_K post padding = 64, total_iteration = 4, overall weights length post-padding = 13824
total_compiled_data.size = 352832
number in buffer entries = 352832, dtype=int8
number output buffer entries = 802816
latency = 0.0017966651916503907
finish resha

len(overall_scale) = 104  number after padding 128-bit interface and parallel interfaces = 26
len(weights_zp) = 104  number after padding 128-bit interface and parallel interfaces = 7
len(weights) = 97344  number after padding 128-bit interface and parallel interfaces = 6084
len(iacts) = 84672  number after padding 128-bit interface and parallel interfaces = 5292
self.K = 104, T_K = 16, overall_K post padding = 112, total_iteration = 7, overall weights length post-padding = 104832
total_compiled_data.size = 190032
number in buffer entries = 190032, dtype=int8
number output buffer entries = 87808
latency = 0.0008713102340698242
finish reshape
executing network 5 layer 25
1, 512, 104, 1, 1, 28, 28, 0, 0, 1, 1, 1, 1, 64, 0.044501472264528275
executing network 5 layer 26
1, 176, 512, 1, 1, 28, 28, 0, 0, 1, 1, 1, 1, 52, 0.15353798866271973
executing network 5 layer 27
1, 176, 176, 3, 3, 28, 28, 1, 1, 1, 1, 1, 1, 65, 0.10692408680915833
padding_x=1
padding_y=1
stride_x=1
stride_y=1
dilation_

len(overall_scale) = 208  number after padding 128-bit interface and parallel interfaces = 52
len(weights_zp) = 208  number after padding 128-bit interface and parallel interfaces = 13
len(weights) = 389376  number after padding 128-bit interface and parallel interfaces = 24336
len(iacts) = 42336  number after padding 128-bit interface and parallel interfaces = 2646
self.K = 208, T_K = 16, overall_K post padding = 208, total_iteration = 13, overall weights length post-padding = 389376
total_compiled_data.size = 432752
number in buffer entries = 432752, dtype=int8
number output buffer entries = 40768
latency = 0.0009268653392791748
finish reshape
executing network 5 layer 47
1, 1024, 208, 1, 1, 14, 14, 0, 0, 1, 1, 1, 1, 63, 0.1437145471572876
executing network 5 layer 48
1, 512, 1024, 1, 1, 14, 14, 0, 0, 1, 1, 1, 1, 84, 0.3120559751987457
executing network 5 layer 49
1, 512, 512, 3, 3, 14, 14, 1, 1, 2, 2, 1, 1, 82, 0.1999092549085617
padding_x=1
padding_y=1
stride_x=2
stride_y=2
dilatio

0.001316969394683838 0.0010275328159332275 0.0017966651916503907 0.0 0.0006485164165496826 0.0 0.0 0.0 0.0006435954570770264 0.0 0.0 0.0006505656242370606 0.0 0.0 0.0011108565330505372 0.0 0.0 0.0030739164352416993 0.0 0.0 0.0 0.0012226462364196776 0.0 0.0 0.0008713102340698242 0.0 0.0 0.002142084836959839 0.0 0.0 0.0043901634216308594 0.0 0.0 0.0 0.0013319289684295655 0.0 0.0 0.00132729172706604 0.0 0.0 0.0009272909164428711 0.0 0.0 0.0009299945831298828 0.0 0.0 0.0009268653392791748 0.0 0.0 0.004925874471664429 0.0 0.0 0.0 0.001781867742538452 0.0 0.0 0.0034138858318328857 0.0 0.0 0.0012042367458343507 0.0 

#### latency of different model###

0.007234785556793213
0.00986584782600403
0.018112189769744873
0.024609420299530026
0.0349132227897644
0.03566405892372131


## Results Analysis

The above script demonstrates the FETHAER accelerator processing different layers from 5 different SubNets (model 1~5 with sizes sorted in the increased fashion) from once-for-all weight-sharing ResNet-50. 
We capture the input activations (iActs), weights, and output activations (oActs) to the path ```/home/xilinx/jupyter_notebooks/qconv2d/CPU_Kernel/``` for ease of reproducing the results.

The per-layer latency of processing all different convolutions layers are shown above.
```
Model 1: 0.0007022202014923095 0.0007643675804138183 0.0 0.00024798870086669924 0.0 0.0 0.0 0.000246429443359375 0.0 0.0 0.000700913667678833 0.0 0.0 0.0 0.0002485334873199463 0.0 0.0 0.0008069634437561035 0.0 0.0 0.0 0.0003100240230560303 0.0 0.0 0.00031036972999572756 0.0 0.0 0.00040453195571899414 0.0 0.0 0.00031578660011291507 0.0 0.0 0.001274811029434204 0.0 0.0 0.0 0.0009018456935882568 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
Model 2: 0.0007112419605255127 0.0009643566608428955 0.0 0.0002244722843170166 0.0 0.0 0.0 0.0003683745861053467 0.0 0.0 0.0006824636459350586 0.0 0.0 0.0 0.0003739726543426514 0.0 0.0 0.000247420072555542 0.0 0.0 0.0009892547130584716 0.0 0.0 0.0 0.00034340262413024903 0.0 0.0 0.0005568742752075195 0.0 0.0 0.00035610079765319825 0.0 0.0 0.00034212946891784665 0.0 0.0 0.0019079363346099854 0.0 0.0 0.0 0.0008983862400054932 0.0 0.0 0.0008994615077972412 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
Model 3: 0.000989212989807129 0.001349184513092041 0.0 0.0004974651336669922 0.0 0.0 0.0 0.0004953086376190185 0.0 0.0 0.00030861139297485354 0.0 0.0 0.00031133413314819334 0.0 0.0 0.0016855370998382568 0.0 0.0 0.0 0.0004963302612304688 0.0 0.0 0.0004954648017883301 0.0 0.0 0.00030939459800720213 0.0 0.0 0.0013552868366241454 0.0 0.0 0.0 0.0007248818874359131 0.0 0.0 0.0007394218444824219 0.0 0.0 0.0007220923900604248 0.0 0.0 0.0007268607616424561 0.0 0.0 0.003824096918106079 0.0 0.0 0.0 0.0015422630310058594 0.0 0.0 0.001539442539215088 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
Model 4: 0.0009895968437194824 0.0007666897773742676 0.0013536489009857177 0.0 0.0004963982105255127 0.0 0.0 0.0 0.0004990267753601075 0.0 0.0 0.0004933476448059082 0.0 0.0 0.0004917895793914795 0.0 0.0 0.003032205104827881 0.0 0.0 0.0 0.0004983377456665039 0.0 0.0 0.000495532751083374 0.0 0.0 0.0003100883960723877 0.0 0.0 0.0022715330123901367 0.0 0.0 0.0 0.000741424560546875 0.0 0.0 0.0010510659217834472 0.0 0.0 0.0010458576679229737 0.0 0.0 0.0007344353199005127 0.0 0.0 0.0038182711601257324 0.0 0.0 0.0 0.0029397130012512205 0.0 0.0 0.001537489891052246 0.0 0.0 0.0010429680347442626 0.0 0.0 0.0 0.0 
Model 5: 0.001320323944091797 0.0010224950313568115 0.0017971324920654297 0.0 0.0006433188915252686 0.0 0.0 0.0 0.0011075043678283691 0.0 0.0 0.0006442499160766601 0.0 0.0 0.0003997683525085449 0.0 0.0 0.0030734097957611085 0.0 0.0 0.0 0.0008840882778167725 0.0 0.0 0.0008867108821868897 0.0 0.0 0.0008839941024780273 0.0 0.0 0.004391485452651977 0.0 0.0 0.0 0.000927811861038208 0.0 0.0 0.002540256977081299 0.0 0.0 0.0013240218162536622 0.0 0.0 0.0013235890865325927 0.0 0.0 0.000926743745803833 0.0 0.0 0.006192008256912232 0.0 0.0 0.0 0.002220135927200317 0.0 0.0 0.0012031257152557372 0.0 0.0 0.0012010478973388673 0.0 
```
The overall latency of different models are
```
model 1: 0.007234785556793213 second
model 2: 0.00986584782600403 second
model 3: 0.018112189769744873 second
model 4: 0.024609420299530026 second
model 5: 0.0349132227897644 second
```